# SLP (NMSE)

This notebook computes the normalized mean square error of atmospheric surface pressure.
It is compared to ERA5 observations, as well as the CESM2 large ensemble and CMIP6 model output.

In [ ]:
import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from cupid_utils.atm import nmse
from cupid_utils.atm import seasonal_climatology_weighted

## Parameters

These variables are set in `config.yml`

In [ ]:
CESM_output_dir = ""
case_name = ""
start_date = ""
end_date = ""
base_case_output_dir = None
ts_dir = None
base_case_name = None
base_start_date = None
base_end_date = None
obs_data_dir = ""
validation_path = ""
regridded_output = False
base_regridded_output = None

In [ ]:
# Want some base case parameter defaults to equal control case values
if base_case_name is not None:
    if base_case_output_dir is None:
        base_case_output_dir = CESM_output_dir

    if base_start_date is None:
        base_start_date = start_date

    if base_end_date is None:
        base_end_date = end_date

    if base_regridded_output is None:
        base_regridded_output = regridded_output
if ts_dir is None:
    ts_dir = CESM_output_dir

## Read in the current case

In [ ]:
def fix_time_dim(dat):
    """CESM2 output sets time as the end of the averaging interval (e.g. January average is midnight on February 1st);
    This function sets the time dimension to the midpoint of the averaging interval.
    Note that CESM3 output sets time to the midpoint already, so this function should not change CESM3 data.
    """
    if "time" not in dat.dims:
        return dat
    if "bounds" not in dat.time.attrs:
        return dat
    time_bounds_avg = dat[dat.time.attrs["bounds"]].mean("nbnd")
    time_bounds_avg.attrs = dat.time.attrs
    dat = dat.assign_coords({"time": time_bounds_avg})
    return xr.decode_cf(dat)

In [ ]:
if regridded_output:
    file_path = f"{ts_dir}/{case_name}/atm/proc/tseries/regrid"
else:
    file_path = f"{ts_dir}/{case_name}/atm/proc/tseries"
print(file_path)

In [ ]:
if base_case_name is not None:
    if base_regridded_output:
        base_file_path = f"{ts_dir}/{base_case_name}/atm/proc/tseries/regrid"
    else:
        base_file_path = f"{ts_dir}/{base_case_name}/atm/proc/tseries"
    print(base_file_path)


################################################################
# Loop over multiple base cases
################################################################
## will have to change base_case_name in config file to base_case_list & change some above lines accordingly to get rid of base_case_name variable
base_case_list = [
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.228",
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.258",
    "b.e30_alpha07c_cesm.B1850C_LTso.ne30_t232_wgx3.232",
]
base_case_nicknames = ["BLT1850_228", "BLT1850_258", "BLT1850_232"]
base_file_paths = []
base_case_info = {}
if base_case_list is not None:
    for base_name in base_case_list:
        if base_regridded_output:
            base_file_path = f"{ts_dir}/{base_name}/atm/proc/tseries/regrid"
        else:
            base_file_path = f"{ts_dir}/{base_name}/atm/proc/tseries"
        # base_file_paths.append(base_file_path)
        base_case_info[base_name] = [base_file_path]
    # print(base_file_paths)

print(base_case_info)

In [ ]:
dat = (
    fix_time_dim(xr.open_mfdataset(f"{file_path}/*PSL*.nc", decode_times=False))
    .sel(time=slice(start_date, end_date))
    .PSL
    / 100.0
)

# Ensure all datasets have the same coordinates as the output data
# (Avoid round-off level differences since all data should be on the same grid)
lon = dat.lon.data
lat = dat.lat.data

if base_case_name is not None:
    base_dat = (
        fix_time_dim(
            xr.open_mfdataset(f"{base_file_path}/*PSL*.nc", decode_times=False)
        )
        .sel(time=slice(base_start_date, base_end_date))
        .assign_coords({"lon": lon, "lat": lat})
        .PSL
        / 100.0
    )

# print('base_file_path:', base_file_path)
# print('base_dat:', base_dat)


################################################################
# Loop over multiple base cases
################################################################
if base_case_list is not None:
    for base_name, base_file_path in base_case_info.items():
        print("base_name:", base_name)
        print("base_file_path:", base_file_path)
        base_dat = (
            fix_time_dim(
                xr.open_mfdataset(f"{base_file_path[0]}/*PSL*.nc", decode_times=False)
            )
            .sel(time=slice(base_start_date, base_end_date))
            .assign_coords({"lon": lon, "lat": lat})
            .PSL
            / 100.0
        )
        base_case_info[base_name].append(base_dat)

print(base_case_info)

In [ ]:
# --Compute seasonal and annual means
dat = seasonal_climatology_weighted(dat).load()

if base_case_name is not None:
    base_dat = seasonal_climatology_weighted(base_dat).load()
    # print('base_dat:', base_dat)

################################################################
# Loop over multiple base cases
################################################################

### base_case_info = {base_name: [base_file_path, base_dat]}

if base_case_list is not None:
    for base_name, base_data in base_case_info.items():
        base_data[1] = seasonal_climatology_weighted(base_data[1]).load()
        # base_case_info[base_name] = base_data[1]

print(base_case_info)

## Read in validation data and other CMIP models for comparison (precomputed)

In [ ]:
# ---ERA5
era5 = xr.open_dataset(
    os.path.join(obs_data_dir, validation_path, "PSL_ERA5.nc")
).assign_coords({"lon": lon, "lat": lat})
era5 = era5 / 100.0  # convert to hPa

# ---CESM2
lens2 = xr.open_dataset(
    os.path.join(obs_data_dir, validation_path, "PSL_LENS2.nc")
).assign_coords({"lon": lon, "lat": lat})
lens2 = lens2 / 100.0  # convert to hPa

# ---CMIP6
modelfiles = sorted(
    glob.glob(f"{os.path.join(obs_data_dir,validation_path)}/CMIP6/*.nc")
)
datcmip6 = [
    xr.open_dataset(ifile).assign_coords({"lon": lon, "lat": lat}).mean("M")
    for ifile in modelfiles
]
datcmip6 = xr.concat(datcmip6, dim="model")
datcmip6 = datcmip6 / 100.0

## Compute the NMSE

In [ ]:
nmse_dat = []
nmse_cesm2 = []
nmse_cmip6 = []
if base_case_name is not None:
    nmse_base_dat = []
else:
    nmse_base_dat = {key: None for key in ["AM", "DJF", "MAM", "JJA", "SON"]}
for ivar in era5.data_vars:
    nmse_dat.append(nmse(era5[ivar], dat[ivar]))
    nmse_cesm2.append(nmse(era5[ivar], lens2[ivar]))
    nmse_cmip6.append(nmse(era5[ivar], datcmip6[ivar]))
    if base_case_name is not None:
        nmse_base_dat.append(nmse(era5[ivar], base_dat[ivar]))

nmse_dat = xr.merge(nmse_dat)
nmse_cesm2 = xr.merge(nmse_cesm2)
nmse_cmip6 = xr.merge(nmse_cmip6)
if base_case_name is not None:
    nmse_base_dat = xr.merge(nmse_base_dat)
print("nmse_base_dat:", nmse_base_dat)
print("\n")

################################################################
# Loop over multiple base cases
################################################################
### base_case_info = {base_name: [base_file_path, base_dat]}

if base_case_list is not None:
    for base_name, base_data in base_case_info.items():
        nmse_base_dat = []
        for ivar in era5.data_vars:
            nmse_base_dat.append(nmse(era5[ivar], base_data[1][ivar]))
        nmse_base_dat = xr.merge(nmse_base_dat)
        base_case_info[base_name].append(nmse_base_dat)
else:
    for base_name, base_data in base_case_info.items():
        nmse_base_dat = {key: None for key in ["AM", "DJF", "MAM", "JJA", "SON"]}
        base_case_info[base_name].append(nmse_base_dat)

print("\n")
print("base_case_info with NMSE values:")
print(base_case_info)
print("\n")

for base_name, base_data in zip(base_case_info.keys(), base_case_nicknames):
    base_case_info[base_name].append(base_data)
print("base_case_info with nicknames:")
print(base_case_info)
print("\n")

## base_case_info = {base_name: [base_file_path, base_dat, nmse_base_dat, nickname]}
## base_case_data = [base_file_path, base_dat, nmse_base_dat]
## base_case_info[base_name][0] = base_file_path
## base_case_info[base_name][1] = base_dat
## base_case_info[base_name][2] = nmse_base_dat

### Set up the plot panel

In [ ]:
def plotnmse(fig, cmip6, cesm2, cesm3, cesm_baseline, x1, x2, y1, y2, titlestr):
    ax = fig.add_axes([x1, y1, x2 - x1, y2 - y1])

    cmip6 = cmip6.sortby(cmip6, ascending=False)
    binedges = np.arange(0, cmip6.size, 1)
    ax.bar(
        binedges,
        cmip6,
        width=1,
        bottom=0,
        edgecolor="black",
        color="gray",
        label="CMIP6",
    )

    ax.plot(cmip6.size + 1, cesm3, "o", color="blue", label="THIS RUN")

    ################################################################
    # Loop over multiple base cases
    ################################################################
    if base_case_list is not None:
        for base_name, base_data in base_case_info.items():
            nmse_baseline = base_data[2][
                cesm_baseline
            ]  ### Maybe do this differently, try to pass it as a better argument not just "AM, DJF, etc."
            if nmse_baseline is not None:
                ax.plot(
                    cmip6.size + 1,
                    nmse_baseline,
                    "x",
                    label=f"BASELINE {base_data[-1]}",
                )
    ################################################################

    # ax.plot(cmip6.size + 1, cesm_baseline, "x", color="red", label="BASELINE")

    ax.fill_between(
        np.arange(0, cmip6.size + 3, 1) - 0.5,
        np.arange(0, cmip6.size + 3, 1) * 0 + np.array(cesm2.min()),
        np.arange(0, cmip6.size + 3, 1) * 0 + np.array(cesm2.max()),
        color="salmon",
        alpha=0.5,
        label="LENS2",
    )

    ax.set_xlim(-0.5, cmip6.size + 2 - 0.5)
    ax.set_xticks([])
    ax.set_ylabel("NMSE", fontsize=14)
    ax.set_title(titlestr, fontsize=16)

    ax.legend()

    return ax

In [ ]:
fig = plt.figure(figsize=(16, 16))

vert_coord = 0.99
fig.text(
    0.5,
    vert_coord,
    "THIS RUN = " + case_name + " " + start_date + " to " + end_date,
    ha="center",
    va="center",
    fontsize=14,
    color="royalblue",
)
vert_coord = vert_coord - 0.015

################################################################
# Loop over multiple base cases
################################################################
for base_name, base_data in base_case_info.items():
    fig.text(
        0.5,
        vert_coord,
        "BASELINE RUN = " + base_name + " " + base_start_date + " to " + base_end_date,
        ha="center",
        va="center",
        fontsize=14,
        color="red",
    )
vert_coord = vert_coord - 0.015
################################################################


# if base_case_name is not None:
#     fig.text(
#         0.5,
#         vert_coord,
#         "BASELINE RUN = "
#         + base_case_name
#         + " "
#         + base_start_date
#         + " to "
#         + base_end_date,
#         ha="center",
#         va="center",
#         fontsize=14,
#         color="red",
#     )
#     vert_coord = vert_coord - 0.015

fig.text(
    0.5,
    vert_coord,
    "Other runs = 1979-01-01 to 2023-12-31",
    ha="center",
    va="center",
    fontsize=14,
)
vert_coord = vert_coord - 0.015

fig.text(
    0.5,
    vert_coord,
    "Validation data = ERA5 1979-01-01 to 2023-12-31",
    ha="center",
    va="center",
    fontsize=14,
)
vert_coord = vert_coord - 0.03

ax = plotnmse(
    fig,
    nmse_cmip6["AM"],
    nmse_cesm2["AM"],
    nmse_dat["AM"],
    "AM",
    0.3,
    0.7,
    vert_coord - 0.16,
    vert_coord,
    "NMSE, SLP, AM",
)
ax = plotnmse(
    fig,
    nmse_cmip6["DJF"],
    nmse_cesm2["DJF"],
    nmse_dat["DJF"],
    "DJF",
    0.05,
    0.45,
    0.57,
    0.72,
    "NMSE, SLP, DJF",
)
ax = plotnmse(
    fig,
    nmse_cmip6["MAM"],
    nmse_cesm2["MAM"],
    nmse_dat["MAM"],
    "MAM",
    0.55,
    0.95,
    0.57,
    0.72,
    "NMSE, SLP, MAM",
)
ax = plotnmse(
    fig,
    nmse_cmip6["JJA"],
    nmse_cesm2["JJA"],
    nmse_dat["JJA"],
    "JJA",
    0.05,
    0.45,
    0.37,
    0.52,
    "NMSE, SLP, JJA",
)
ax = plotnmse(
    fig,
    nmse_cmip6["SON"],
    nmse_cesm2["SON"],
    nmse_dat["SON"],
    "SON",
    0.55,
    0.95,
    0.37,
    0.52,
    "NMSE, SLP, SON",
)